In [19]:
import numpy as np
import sys
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.python.keras.utils import np_utils
from keras.optimizers import SGD
from keras.datasets import mnist
import pandas as pd
np.random.seed(42)  


In [20]:

DEFAULT_LEARNING_RATE = 0.01
DEFAULT_NUMBER_HIDDEN = 2
DEFAULT_BATCH_SIZE = 32
scores = {}


In [21]:
(X_train_1, train_labels_1), (X_test, test_labels_1) = mnist.load_data()

In [22]:
# Load pre-shuffled MNIST data into train and test sets
X_train_1 = X_train_1.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train_1 = X_train_1.astype('float32')
X_test = X_test.astype('float32')

In [23]:
# Preprocess input data
X_train_1 /= 255
X_test /= 255

In [24]:
# Divides the dataset into train and validation sets
X_valid = X_train_1[50000:60000]
X_train = X_train_1[:50000]
print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'validation samples')

50000 train samples
10000 validation samples


In [25]:
# Preprocess class labels
train_labels = np_utils.to_categorical(train_labels_1, 10)
test_labels = np_utils.to_categorical(test_labels_1, 10)
valid_labels = train_labels[50000:60000]
train_labels = train_labels[:50000]

In [26]:
model = Sequential()

model.add(Dense(64, input_dim=784))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

In [27]:
for batch_num in [1,2,4,8,16,32,64,128]: 
               
    model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=DEFAULT_LEARNING_RATE), metrics=['accuracy'])
    model.fit(X_train, train_labels, epochs= 5, batch_size=batch_num, verbose=1, validation_data=(X_valid, valid_labels))
    score = model.evaluate(X_test, test_labels, batch_size=batch_num, verbose=1)
    scores[(batch_num,DEFAULT_NUMBER_HIDDEN,DEFAULT_LEARNING_RATE)] = (score[1],score[0])

for hidden_num in [1,2,4,8,16,32,64,128]:
    model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=DEFAULT_LEARNING_RATE), metrics=['accuracy'])
    model.fit(X_train, train_labels, epochs= 5, batch_size=DEFAULT_BATCH_SIZE, verbose=1, validation_data=(X_valid, valid_labels))
    score = model.evaluate(X_test, test_labels, batch_size=DEFAULT_BATCH_SIZE, verbose=1)
    scores[(DEFAULT_BATCH_SIZE,hidden_num,DEFAULT_LEARNING_RATE)] = (score[1],score[0])


for learning_r in [0.0001,0.001,0.01,0.1,1]:

    model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=learning_r), metrics=['accuracy'])
    model.fit(X_train, train_labels, epochs= 5, batch_size=DEFAULT_BATCH_SIZE, verbose=1, validation_data=(X_valid, valid_labels))
    score = model.evaluate(X_test, test_labels, batch_size=DEFAULT_BATCH_SIZE, verbose=1)
    scores[(DEFAULT_BATCH_SIZE,DEFAULT_NUMBER_HIDDEN,learning_r)] = (score[1],score[0])

np.save('scores.npy', scores)

df = pd.DataFrame.from_dict(scores, orient='index')
df.to_excel('results.xlsx')



Epoch 1/5
50000/50000 [==============================] - 98s 2ms/step - loss: 0.2676 - accuracy: 0.9185 - val_loss: 0.1812 - val_accuracy: 0.9451
Epoch 2/5
50000/50000 [==============================] - 95s 2ms/step - loss: 0.1357 - accuracy: 0.9592 - val_loss: 0.1131 - val_accuracy: 0.9657
Epoch 3/5
50000/50000 [==============================] - 99s 2ms/step - loss: 0.1047 - accuracy: 0.9678 - val_loss: 0.1235 - val_accuracy: 0.9627
Epoch 4/5
50000/50000 [==============================] - 91s 2ms/step - loss: 0.0827 - accuracy: 0.9749 - val_loss: 0.1267 - val_accuracy: 0.9692
Epoch 5/5
10000/10000 [==============================] - 13s 1ms/step - loss: 0.1029 - accuracy: 0.9719
Epoch 1/5
25000/25000 [==============================] - 48s 2ms/step - loss: 0.0381 - accuracy: 0.9877 - val_loss: 0.0961 - val_accuracy: 0.9742
Epoch 2/5
25000/25000 [==============================] - 48s 2ms/step - loss: 0.0269 - accuracy: 0.9916 - val_loss: 0.0998 - val_accuracy: 0.9761
Epoch 3/5
25000/2500

NameError: name 'learning_rate' is not defined